# Fine-tuning para Geração de SQL

Este notebook realiza o fine-tuning do modelo Mistral 7B para gerar consultas SQL a partir de perguntas em português.

## Instalação das Dependências

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

## Carregamento do Modelo

In [ ]:
from unsloth import FastLanguageModel
import torch

# Configurações do modelo
max_seq_length = 2048  # Comprimento máximo da sequência
dtype = None  # Detecção automática do tipo de dados
load_in_4bit = True  # Usar quantização 4-bit para reduzir uso de memória

# Carregar o modelo
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

## Configuração do LoRA

In [ ]:
# Configurar LoRA para treinamento eficiente
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,  # Rank do LoRA
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                     "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407
)

## Preparação dos Dados

In [ ]:
import json
from datasets import Dataset

# Carregar o dataset
with open('delfos_fine_tunning.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Converter para o formato do Hugging Face
dataset = Dataset.from_list(data)

## Configuração do Treinamento

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# Função de formatação para o dataset
def formatting_func(example):
    conversations = example['conversations']
    formatted_text = ""
    for message in conversations:
        if message['role'] == 'system':
            formatted_text += f"<|im_start|>system\n{message['content']}<|im_end|>\n"
        elif message['role'] == 'user':
            formatted_text += f"<|im_start|>user\n{message['content']}<|im_end|>\n"
        elif message['role'] == 'assistant':
            formatted_text += f"<|im_start|>assistant\n{message['content']}<|im_end|>\n"
    return formatted_text

# Configurar o treinador
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    formatting_func = formatting_func,
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

## Iniciar o Treinamento

In [ ]:
# Iniciar o treinamento
trainer.train()

## Salvar o Modelo

In [ ]:
# Salvar o modelo treinado
model.save_pretrained("sql_model")
tokenizer.save_pretrained("sql_model")

## Testar o Modelo

In [ ]:
def generate_sql(question):
    prompt = f"<|im_start|>system\nVocê é um assistente especializado em gerar consultas SQL a partir de perguntas em português. Sua função é traduzir perguntas em linguagem natural para consultas SQL válidas, considerando as seguintes tabelas:\n\n- cliente (id, nome, email)\n- endereco (id, cliente_id, rua, cidade, estado, cep)\n- produto (id, nome, preco)\n- venda (id, cliente_id, produto_id, quantidade, total)\n\nVocê deve gerar apenas a consulta SQL, sem explicações adicionais.<|im_end|>\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n"

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        temperature=0.1,
        top_p=0.95,
        repetition_penalty=1.1
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Testar com algumas perguntas
test_questions = [
    "Quais são os nomes e e-mails dos clientes?",
    "Liste os endereços de todos os clientes que moram em Minas Gerais.",
    "Mostre os clientes e os produtos que eles compraram."
]

for question in test_questions:
    print(f"\nPergunta: {question}")
    print(f"SQL: {generate_sql(question)}")